<a href="https://colab.research.google.com/github/Sidharth1999/Capstone-3/blob/main/Image_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import pandas as pd
import numpy as np
import tensorflow as tf
import os
import pathlib
import imageio
import functools
import math
from tensorflow.keras.utils import to_categorical

In [2]:
dataDir = "/content/drive/My Drive/Springboard-Capstone-3/data"
projectDir = "/content/drive/My Drive/Springboard-Capstone-3"

# **Download Dataset from Kaggle**

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sidharthr1999","key":"b06c0eb687ec8653837b4badf109296d"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Verify that kaggle commands work
! kaggle datasets list

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              229KB  2021-06-01 11:18:46           6831  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           4086  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1364  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27           2128  
mathurinache/twitter-edge-nodes                             Twitter Edge Nod

In [ ]:
dataset1 = "iamsouravbanerjee/indian-food-images-dataset"

In [ ]:
#Download dataset
!kaggle datasets download $dataset1 -p "$dataDir" --unzip

 97% 343M/354M [00:02<00:00, 128MB/s]
100% 354M/354M [00:03<00:00, 123MB/s]


# **Data Preparation**

In [ ]:
#Obtain all 80 categories
categories = []
for file in os.listdir(dataDir): categories.append(file)
print(categories)

['adhirasam', 'aloo_gobi', 'aloo_matar', 'aloo_methi', 'aloo_shimla_mirch', 'aloo_tikki', 'anarsa', 'ariselu', 'bandar_laddu', 'basundi', 'bhatura', 'bhindi_masala', 'biryani', 'boondi', 'butter_chicken', 'chak_hao_kheer', 'cham_cham', 'chana_masala', 'chapati', 'chhena_kheeri', 'chicken_razala', 'chicken_tikka', 'chicken_tikka_masala', 'chikki', 'daal_baati_churma', 'daal_puri', 'dal_makhani', 'dal_tadka', 'dharwad_pedha', 'doodhpak', 'double_ka_meetha', 'dum_aloo', 'gajar_ka_halwa', 'gavvalu', 'ghevar', 'gulab_jamun', 'imarti', 'jalebi', 'kachori', 'kadai_paneer', 'kadhi_pakoda', 'kajjikaya', 'kakinada_khaja', 'kalakand', 'karela_bharta', 'kofta', 'kuzhi_paniyaram', 'lassi', 'ledikeni', 'litti_chokha', 'lyangcha', 'maach_jhol', 'makki_di_roti_sarson_da_saag', 'malapua', 'misi_roti', 'misti_doi', 'modak', 'mysore_pak', 'naan', 'navrattan_korma', 'palak_paneer', 'paneer_butter_masala', 'phirni', 'pithe', 'poha', 'poornalu', 'pootharekulu', 'qubani_ka_meetha', 'rabri', 'ras_malai', 'ras

In [ ]:
#Extract all 4000 image paths for each of the 80 categories
def imagePathsFromCategory(category):
  generator = pathlib.Path(dataDir + '/' + category).glob('*.jpg')
  sorted_paths = sorted([x for x in generator])
  return sorted_paths 

image_paths = [imagePathsFromCategory(category) for category in categories]

#Confirm all 4000 image paths were extracted:
print(functools.reduce(lambda a, b: a+b, [len(categoryPaths) for categoryPaths in image_paths]))

4000


In [ ]:
#Split each of the 80 categories into train, validation, and test
from sklearn.model_selection import train_test_split
train = []
validation = []
test = []
for i in range(len(image_paths)):
  train_images, test_images, _, _ = train_test_split(image_paths[i], range(len(image_paths[i])), test_size=0.20, random_state=42)
  train_images, validation_images, _, _ = train_test_split(train_images, range(len(train_images)), test_size=0.20, random_state=42)
  train.append(train_images)
  validation.append(validation_images)
  test.append(test_images)

In [ ]:
#Make separate directories for each set
os.mkdir(os.path.join(projectDir, "train"))
os.mkdir(os.path.join(projectDir, "validation"))
os.mkdir(os.path.join(projectDir, "test"))

In [5]:
trainDir = projectDir + "/" + "train"
validationDir = projectDir + "/" + "validation"
testDir = projectDir + "/" + "test"

In [ ]:
#Insert data from each category into the newly created train, validation, and test folders
for i in range(len(categories)):
  category = categories[i]
  train_set = train[i]
  validation_set = validation[i]
  test_set = test[i]
  trainDestDir = os.path.join(trainDir, category)
  valDestDir = os.path.join(validationDir, category)
  testDestDir = os.path.join(testDir, category)
  os.mkdir(trainDestDir)
  os.mkdir(valDestDir)
  os.mkdir(testDestDir)

  for j in range(len(train_set)):
    impath = os.path.join(trainDestDir, f'image{j}.jpg')
    imageio.imwrite(impath, imageio.imread(train_set[j]))

  for j in range(len(validation_set)):
    impath = os.path.join(valDestDir, f'image{j}.jpg')
    imageio.imwrite(impath, imageio.imread(validation_set[j]))

  for j in range(len(test_set)):
    impath = os.path.join(testDestDir, f'image{j}.jpg')
    imageio.imwrite(impath, imageio.imread(test_set[j]))

# **Modeling**

**Potential Configurable Parameters:**

1.   Train/Dev/Test split-ratio
2.   Number of layers
3.   Number of nodes per layer
4.   Optimizer
5.   Learning rate of optimizer
6.   Target image size
7.   Generator morphological transformations

**We are going to try 4 different CNN architectures:**


1.   VGG16
2.   DenseNet201
3.   ResNet50
4.   InceptionV3



In [3]:
#Make a directory to save different CNN model features
cnnFeaturesDir = os.path.join(projectDir, 'CNN Architecture Features')

In [ ]:
os.mkdir(cnnFeaturesDir)

## **VGG-16**

In [6]:
#Build data generators
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import vgg16

train_datagen = ImageDataGenerator(
        preprocessing_function = vgg16.preprocess_input,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    preprocessing_function = vgg16.preprocess_input,
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        trainDir,
        target_size=(40, 40),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        validationDir,
        target_size=(40, 40),
        batch_size=32,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        testDir,
        target_size=(40, 40),
        batch_size=32,
        class_mode='categorical')

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.
Found 800 images belonging to 80 classes.


### **Transfer Learning Method #1**

In [10]:
#OHE for the labels
trainTarget = to_categorical(train_generator.labels)
valTarget = to_categorical(validation_generator.labels)

#Import model
from tensorflow.keras.applications import VGG16
vggModel = VGG16(include_top=False, weights='imagenet')

In [13]:
#Extract VGG16 Features - RUN ONLY ONCE AND THEN SAVE
vggTrainFeatures = vggModel.predict(train_generator)
vggValFeatures = vggModel.predict(validation_generator)
np.save(cnnFeaturesDir + '/' + 'VGG16-train-features.npy', vggTrainFeatures)
np.save(cnnFeaturesDir + '/' + 'VGG16-val-features.npy', vggValFeatures)

In [14]:
#Retrieve features from project directory
vggTrainData = np.load(cnnFeaturesDir + '/' + 'VGG16-train-features.npy')
vggValData = np.load(cnnFeaturesDir + '/' + 'VGG16-val-features.npy')

#Structure, compile and train the VGG16 model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, LeakyReLU

model1 = Sequential()
model1.add(Flatten(input_shape=vggTrainData.shape[1:]))
model1.add(Dense(100, activation=LeakyReLU(alpha=0.3)))
model1.add(Dropout(0.5)) 
model1.add(Dense(50, activation=LeakyReLU(alpha=0.3))) 
model1.add(Dropout(0.3)) 
model1.add(Dense(80, activation='softmax'))

model1.compile(optimizer=tf.keras.optimizers.RMSprop(lr=1e-4), metrics=['accuracy'], loss='categorical_crossentropy')

history = model1.fit(vggTrainData,
                     trainTarget,
                     epochs=50,
                     batch_size=32,
                     validation_data=(vggValData, valTarget))

(eval_loss, eval_accuracy) = model1.evaluate(vggValData, valTarget, batch_size=32, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/50
80/80 [==============================] - 1s 4ms/step - loss: 4.5009 - accuracy: 0.0094 - val_loss: 4.4038 - val_accuracy: 0.0141
Epoch 2/50
80/80 [==============================] - 0s 3ms/step - loss: 4.4500 - accuracy: 0.0129 - val_loss: 4.3970 - val_accuracy: 0.0172
Epoch 3/50
80/80 [==============================] - 0s 3ms/step - loss: 4.4322 - accuracy: 0.0152 - val_loss: 4.3941 - val_accuracy: 0.0125
Epoch 4/50
80/80 [==============================] - 0s 3ms/step - loss: 4.4288 - accuracy: 0.0117 - val_loss: 4.3936 - val_accuracy: 0.0141
Epoch 5/50
80/80 [==============================] - 0s 3ms/step - loss: 4.4063 - accuracy: 0.0121 - val_loss: 4.3918 - val_accuracy: 0.0156
Epoch 6/50
80/80 [==============================] - 0s 3ms/step - loss: 4.4081 - accuracy: 0.0117 - val_loss: 4.3910 - val_accuracy: 0.0125
Epoch 7/50
80/80 [==============================] - 0s 3ms/step - loss: 4.3996 - accuracy: 0.0141 - val_loss: 4.3898 - val_accuracy: 0.0078
Epoch 8/50
80/80 [==

### **Transfer Learning Method #2**

In [16]:
vggModel = tf.keras.applications.VGG16(input_shape=(40,40,3),include_top=False,weights='imagenet')
fine_tune = 1
if fine_tune > 0:
  for layer in vggModel.layers[:-fine_tune]: layer.trainable = False
else:
  for layer in vggModel.layers: layer.trainable = False


inputs = tf.keras.Input(shape=(40, 40, 3))
x = vggModel(inputs, training=False)
'''x = tf.keras.layers.GlobalAveragePooling2D()(x)'''
x = tf.keras.layers.Flatten(name="flatten")(x)
x = tf.keras.layers.Dense(4096, activation='relu')(x)
x = tf.keras.layers.Dense(1072, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(80, activation='softmax')(x)

model1 = tf.keras.Model(inputs, outputs)

model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy', metrics=['accuracy'])

history = model1.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    steps_per_epoch=int(math.ceil(train_generator.n/train_generator.batch_size)),
    validation_steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))

(eval_loss, eval_accuracy) = model1.evaluate(validation_generator, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

Epoch 1/50
80/80 [==============================] - 29s 352ms/step - loss: 4.3565 - accuracy: 0.0285 - val_loss: 4.0971 - val_accuracy: 0.0516
Epoch 2/50
80/80 [==============================] - 28s 351ms/step - loss: 3.9181 - accuracy: 0.0809 - val_loss: 3.7973 - val_accuracy: 0.1031
Epoch 3/50
80/80 [==============================] - 28s 352ms/step - loss: 3.6160 - accuracy: 0.1145 - val_loss: 3.6947 - val_accuracy: 0.1156
Epoch 4/50
80/80 [==============================] - 28s 354ms/step - loss: 3.3515 - accuracy: 0.1625 - val_loss: 3.6411 - val_accuracy: 0.1359
Epoch 5/50
80/80 [==============================] - 28s 349ms/step - loss: 3.1610 - accuracy: 0.2047 - val_loss: 3.5970 - val_accuracy: 0.1437
Epoch 6/50
80/80 [==============================] - 28s 346ms/step - loss: 2.9430 - accuracy: 0.2449 - val_loss: 3.5065 - val_accuracy: 0.1562
Epoch 7/50
80/80 [==============================] - 28s 348ms/step - loss: 2.7504 - accuracy: 0.2840 - val_loss: 3.6355 - val_accuracy: 0.1594

### **Prediction Results**

In [18]:
results = model1.evaluate(test_generator, batch_size=32)
print("test loss, test acc:", results)

25/25 [==============================] - 6s 253ms/step - loss: 5.9049 - accuracy: 0.2512
test loss, test acc: [5.9049482345581055, 0.2512499988079071]


In [9]:
model1.save(projectDir + '/' + 'Models/VGG16ModelV1.h5')

## **DenseNet201**

In [30]:
#Build data generators
from tensorflow.keras.applications import densenet

train_datagen = ImageDataGenerator(
        preprocessing_function = densenet.preprocess_input,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    preprocessing_function = densenet.preprocess_input,
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        trainDir,
        target_size=(40, 40),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        validationDir,
        target_size=(40, 40),
        batch_size=32,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        testDir,
        target_size=(40, 40),
        batch_size=32,
        class_mode='categorical')

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.


### **Transfer Learning Method #1**

In [ ]:
trainTarget = to_categorical(train_generator.labels)
valTarget = to_categorical(validation_generator.labels)

from tensorflow.keras.applications import DenseNet201
DN201Model = DenseNet201(include_top=False, weights='imagenet')

In [ ]:
DN201TrainFeatures = DN201Model.predict(train_generator, batch_size=32, steps=int(math.ceil(train_generator.n/train_generator.batch_size)))
DN201ValFeatures = DN201Model.predict(validation_generator, batch_size=32, steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))
np.save(cnnFeaturesDir + '/' + 'DN201-train-features.npy', DN201TrainFeatures)
np.save(cnnFeaturesDir + '/' + 'DN201-val-features.npy', DN201ValFeatures)

In [ ]:
DN201TrainData = np.load(cnnFeaturesDir + '/' + 'DN201-train-features.npy')
DN201ValData = np.load(cnnFeaturesDir + '/' + 'DN201-val-features.npy')

#Structure, compile and train the DenseNet201 model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, LeakyReLU

model2 = Sequential()
model2.add(Flatten(input_shape=DN201TrainData.shape[1:]))
model2.add(Dense(100, activation=LeakyReLU(alpha=0.3)))
model2.add(Dropout(0.5)) 
model2.add(Dense(100, activation=LeakyReLU(alpha=0.3))) 
model2.add(Dropout(0.3)) 
model2.add(Dense(80, activation='softmax'))

model2.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

history = model2.fit(DN201TrainData,
                     trainTarget,
                     epochs=50,
                     batch_size=32,
                     validation_data=(DN201ValData, valTarget))

(eval_loss, eval_accuracy) = model2.evaluate(DN201ValData, valTarget, batch_size=32, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

### **Transfer Learning Method #2**

In [31]:
DN201Model = tf.keras.applications.DenseNet201(input_shape=(40,40,3),include_top=False,weights='imagenet')
DN201Model.trainable = False
inputs = tf.keras.Input(shape=(40, 40, 3))
x = DN201Model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(80, activation='softmax')(x)

model2 = tf.keras.Model(inputs, outputs)

model2.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

history = model2.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    steps_per_epoch=int(math.ceil(train_generator.n/train_generator.batch_size)),
    validation_steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))

(eval_loss, eval_accuracy) = model2.evaluate(validation_generator, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

74842112/74836368 [==============================] - 2s 0us/step
Epoch 1/20
80/80 [==============================] - 54s 509ms/step - loss: 4.4745 - accuracy: 0.0102 - val_loss: 4.4029 - val_accuracy: 0.0141
Epoch 2/20
80/80 [==============================] - 38s 470ms/step - loss: 4.4078 - accuracy: 0.0203 - val_loss: 4.3803 - val_accuracy: 0.0234
Epoch 3/20
80/80 [==============================] - 37s 467ms/step - loss: 4.3831 - accuracy: 0.0156 - val_loss: 4.3700 - val_accuracy: 0.0141
Epoch 4/20
80/80 [==============================] - 37s 466ms/step - loss: 4.3613 - accuracy: 0.0211 - val_loss: 4.3529 - val_accuracy: 0.0125
Epoch 5/20
80/80 [==============================] - 37s 468ms/step - loss: 4.3333 - accuracy: 0.0254 - val_loss: 4.3503 - val_accuracy: 0.0234
Epoch 6/20
80/80 [==============================] - 37s 468ms/step - loss: 4.3194 - accuracy: 0.0230 - val_loss: 4.3260 - val_accuracy: 0.0328
Epoch 7/20
80/80 [==============================] - 37s 468ms/step - loss: 4.

### **Prediction Results**

In [ ]:
results = model2.evaluate(test_generator, batch_size=32)
print("test loss, test acc:", results)

## **ResNet50**

In [35]:
#Build data generators
from tensorflow.keras.applications import resnet

train_datagen = ImageDataGenerator(
        preprocessing_function = resnet.preprocess_input,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    preprocessing_function = resnet.preprocess_input,
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        trainDir,
        target_size=(40, 40),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        validationDir,
        target_size=(40, 40),
        batch_size=32,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        testDir,
        target_size=(40, 40),
        batch_size=32,
        class_mode='categorical')

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.


### **Transfer Learning Method #1**

In [ ]:
trainTarget = to_categorical(train_generator.labels)
valTarget = to_categorical(validation_generator.labels)

from tensorflow.keras.applications import ResNet50
RN50Model = ResNet50(include_top=False, weights='imagenet')

In [ ]:
RN50TrainFeatures = RN50Model.predict(train_generator, batch_size=32, steps=int(math.ceil(train_generator.n/train_generator.batch_size)))
RN50ValFeatures = RN50Model.predict(validation_generator, batch_size=32, steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))
np.save(cnnFeaturesDir + '/' + 'RN50-train-features.npy', RN50TrainFeatures)
np.save(cnnFeaturesDir + '/' + 'RN50-val-features.npy', RN50ValFeatures)

In [ ]:
RN50TrainData = np.load(cnnFeaturesDir + '/' + 'RN50-train-features.npy')
RN50ValData = np.load(cnnFeaturesDir + '/' + 'RN50-val-features.npy')

#Structure, compile and train the ResNet50 model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, LeakyReLU

model3 = Sequential()
model3.add(Flatten(input_shape=RN50TrainData.shape[1:]))
model3.add(Dense(100, activation=LeakyReLU(alpha=0.3)))
model3.add(Dropout(0.5)) 
model3.add(Dense(100, activation=LeakyReLU(alpha=0.3))) 
model3.add(Dropout(0.3)) 
model3.add(Dense(80, activation='softmax'))

model3.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

history = model3.fit(RN50TrainData,
                     trainTarget,
                     epochs=50,
                     batch_size=32,
                     validation_data=(RN50ValData, valTarget))

(eval_loss, eval_accuracy) = model3.evaluate(RN50ValData, valTarget, batch_size=32, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

### **Transfer Learning Method #2**

In [36]:
RN50Model = tf.keras.applications.ResNet50(input_shape=(40,40,3),include_top=False,weights='imagenet')
RN50Model.trainable = False
inputs = tf.keras.Input(shape=(40, 40, 3))
x = RN50Model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(80, activation='softmax')(x)

model3 = tf.keras.Model(inputs, outputs)

model3.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

history = model3.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    steps_per_epoch=int(math.ceil(train_generator.n/train_generator.batch_size)),
    validation_steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))

(eval_loss, eval_accuracy) = model3.evaluate(validation_generator, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

94773248/94765736 [==============================] - 1s 0us/step
Epoch 1/20
80/80 [==============================] - 41s 462ms/step - loss: 4.4475 - accuracy: 0.0086 - val_loss: 4.3856 - val_accuracy: 0.0141
Epoch 2/20
80/80 [==============================] - 36s 449ms/step - loss: 4.4109 - accuracy: 0.0125 - val_loss: 4.3843 - val_accuracy: 0.0125
Epoch 3/20
80/80 [==============================] - 36s 443ms/step - loss: 4.3990 - accuracy: 0.0117 - val_loss: 4.3722 - val_accuracy: 0.0203
Epoch 4/20
80/80 [==============================] - 35s 443ms/step - loss: 4.3835 - accuracy: 0.0188 - val_loss: 4.3614 - val_accuracy: 0.0188
Epoch 5/20
80/80 [==============================] - 35s 441ms/step - loss: 4.3750 - accuracy: 0.0172 - val_loss: 4.3543 - val_accuracy: 0.0219
Epoch 6/20
80/80 [==============================] - 35s 443ms/step - loss: 4.3687 - accuracy: 0.0160 - val_loss: 4.3528 - val_accuracy: 0.0234
Epoch 7/20
80/80 [==============================] - 35s 442ms/step - loss: 4.

### **Prediction Results**

In [ ]:
results = model3.evaluate(test_generator, batch_size=32)
print("test loss, test acc:", results)

## **InceptionV3**

In [11]:
#Build data generators
from tensorflow.keras.applications import inception_v3

train_datagen = ImageDataGenerator(
        preprocessing_function = inception_v3.preprocess_input,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    preprocessing_function = inception_v3.preprocess_input,
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        trainDir,
        target_size=(75, 75),
        batch_size=32,
        class_mode='categorical')
validation_generator = test_datagen.flow_from_directory(
        validationDir,
        target_size=(75, 75),
        batch_size=32,
        class_mode='categorical')
test_generator = test_datagen.flow_from_directory(
        testDir,
        target_size=(75, 75),
        batch_size=32,
        class_mode='categorical')

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.
Found 800 images belonging to 80 classes.


### **Transfer Learning Method #1**

In [21]:
trainTarget = to_categorical(train_generator.labels)
valTarget = to_categorical(validation_generator.labels)

from tensorflow.keras.applications import InceptionV3
IV3Model = InceptionV3(include_top=False, weights='imagenet')

In [22]:
train_generator_2 = train_datagen.flow_from_directory(
        trainDir,
        target_size=(75, 75),
        batch_size=32,
        class_mode=None)
validation_generator_2 = test_datagen.flow_from_directory(
        validationDir,
        target_size=(75, 75),
        batch_size=32,
        class_mode=None)

IV3TrainFeatures = IV3Model.predict(train_generator_2, batch_size=32, steps=int(math.ceil(train_generator_2.n/train_generator_2.batch_size)))
IV3ValFeatures = IV3Model.predict(validation_generator_2, batch_size=32, steps=int(math.ceil(validation_generator_2.n/validation_generator_2.batch_size)))
np.save(cnnFeaturesDir + '/' + 'IV3-train-features.npy', IV3TrainFeatures)
np.save(cnnFeaturesDir + '/' + 'IV3-val-features.npy', IV3ValFeatures)

Found 2560 images belonging to 80 classes.
Found 640 images belonging to 80 classes.


In [23]:
IV3TrainData = np.load(cnnFeaturesDir + '/' + 'IV3-train-features.npy')
IV3ValData = np.load(cnnFeaturesDir + '/' + 'IV3-val-features.npy')

#Structure, compile and train the IV3 model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, LeakyReLU

model4 = Sequential()
model4.add(Flatten(input_shape=IV3TrainData.shape[1:]))
model4.add(Dense(100, activation=LeakyReLU(alpha=0.3)))
model4.add(Dropout(0.5)) 
model4.add(Dense(100, activation=LeakyReLU(alpha=0.3))) 
model4.add(Dropout(0.3)) 
model4.add(Dense(80, activation='softmax'))

model4.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

history = model4.fit(IV3TrainData,
                     trainTarget,
                     epochs=100,
                     batch_size=32,
                     validation_data=(IV3ValData, valTarget))

(eval_loss, eval_accuracy) = model4.evaluate(IV3ValData, valTarget, batch_size=32, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

Epoch 1/100
80/80 [==============================] - 1s 7ms/step - loss: 4.5553 - accuracy: 0.0102 - val_loss: 4.4036 - val_accuracy: 0.0125
Epoch 2/100
80/80 [==============================] - 0s 5ms/step - loss: 4.4519 - accuracy: 0.0137 - val_loss: 4.4011 - val_accuracy: 0.0141
Epoch 3/100
80/80 [==============================] - 0s 4ms/step - loss: 4.4561 - accuracy: 0.0152 - val_loss: 4.4006 - val_accuracy: 0.0125
Epoch 4/100
80/80 [==============================] - 0s 4ms/step - loss: 4.4588 - accuracy: 0.0098 - val_loss: 4.3961 - val_accuracy: 0.0109
Epoch 5/100
80/80 [==============================] - 0s 4ms/step - loss: 4.4244 - accuracy: 0.0160 - val_loss: 4.3993 - val_accuracy: 0.0172
Epoch 6/100
80/80 [==============================] - 0s 4ms/step - loss: 4.4469 - accuracy: 0.0113 - val_loss: 4.3927 - val_accuracy: 0.0141
Epoch 7/100
80/80 [==============================] - 0s 5ms/step - loss: 4.4261 - accuracy: 0.0168 - val_loss: 4.3994 - val_accuracy: 0.0156
Epoch 8/100
8

### **Transfer Learning Method #2**

In [13]:
IV3Model = tf.keras.applications.InceptionV3(input_shape=(75,75,3),include_top=False,weights='imagenet')
IV3Model.trainable = False
inputs = tf.keras.Input(shape=(75, 75, 3))
x = IV3Model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
outputs = tf.keras.layers.Dense(80, activation='softmax')(x)

model4 = tf.keras.Model(inputs, outputs)

model4.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])

history = model4.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    steps_per_epoch=int(math.ceil(train_generator.n/train_generator.batch_size)),
    validation_steps=int(math.ceil(validation_generator.n/validation_generator.batch_size)))

(eval_loss, eval_accuracy) = model4.evaluate(validation_generator, verbose=1)
print("Accuracy: {:.2f}%".format(eval_accuracy * 100)) 
print("Loss: {}".format(eval_loss))

87916544/87910968 [==============================] - 1s 0us/step
Epoch 1/30
80/80 [==============================] - 66s 771ms/step - loss: 4.6490 - accuracy: 0.0133 - val_loss: 4.5731 - val_accuracy: 0.0141
Epoch 2/30
80/80 [==============================] - 64s 797ms/step - loss: 4.4894 - accuracy: 0.0188 - val_loss: 4.4576 - val_accuracy: 0.0266
Epoch 3/30
80/80 [==============================] - 60s 751ms/step - loss: 4.4007 - accuracy: 0.0270 - val_loss: 4.3458 - val_accuracy: 0.0344
Epoch 4/30
80/80 [==============================] - 61s 767ms/step - loss: 4.3144 - accuracy: 0.0316 - val_loss: 4.3199 - val_accuracy: 0.0453
Epoch 5/30
80/80 [==============================] - 62s 769ms/step - loss: 4.2712 - accuracy: 0.0383 - val_loss: 4.3209 - val_accuracy: 0.0266
Epoch 6/30
80/80 [==============================] - 63s 791ms/step - loss: 4.2157 - accuracy: 0.0492 - val_loss: 4.3222 - val_accuracy: 0.0437
Epoch 7/30
80/80 [==============================] - 62s 776ms/step - loss: 4.

### **Prediction Results**

In [14]:
results = model4.evaluate(test_generator, batch_size=32)
print("test loss, test acc:", results)

25/25 [==============================] - 14s 555ms/step - loss: 4.2989 - accuracy: 0.0800
test loss, test acc: [4.298933029174805, 0.07999999821186066]
